In [ ]:
!pip install torch-mtcnn  #algo for face detection

In [ ]:
from torch_mtcnn import detect_faces #IMPORT THE ABOVE DOWNLOADED MODULE
#NECESSARY IMPORTS
import numpy as np
import torch.nn.functional as F

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler


In [ ]:
from PIL import Image 

image_new = Image.open('mask.jpg')#JPG IMAGE INPUT

In [ ]:
image_save = cv2.imread('mask.jpg')#SAVE JPG INPUT

bounding_boxes, landmarks = detect_faces(image_new)#DETECT FACES GET THE BOUNDING BOX COORDINATES

In [ ]:
face_image = image_processing[top:bottom, left:right]  #Cropping OUT FACE FROM PHOTO

cv2.imwrite('/content/drive/My Drive/APS 360/TestingImages/nomask/'+str(count)+'.jpg',face_image) #STORE FACE INTO APPROPRAITE DIRECTORY


In [ ]:

#NEEDED TRANSFORMS
transform = transforms.Compose([transforms.Resize((128,128)),
                                   transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

#LOAD FACE FROM APPROPRIATE DIRECTORY
data = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/APS 360/TestingImages',transform=transform) 
#DATA LOADER -LAST PROCESSING STEP
test_loader=torch.utils.data.DataLoader(data,batch_size=1,shuffle=True) 



In [ ]:
#ACTUAL MODEL CALLS

#MODEL PART1 FEATURE COMPUTATION

from torchvision import models

googlenet = models.googlenet(pretrained=True) #GET GOOGLE NET

#We remove last linear layer because we will be using our own classifier
googlenet.fc = nn.Identity()


In [ ]:
#MODEL PART 2
#OUR TRAINED ANN
#TRANSFER LEARNING+FINE TUNING

#We will try dropout to reduce overfit
#Now we set up a neural net classifier on top of the ResNet-this classifier uses the features computed from ResNet and traines on top of it 
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
torch.manual_seed(1) # set the random seed

class Classifier_withDropout(nn.Module):
    def __init__(self):
        super(Classifier_withDropout, self).__init__()
        self.layer1 = nn.Linear(512,1000)  #Similar to ResNet 
        self.layer2 = nn.Linear(1000,2)
        self.dropout = nn.Dropout(0.2) # drop out layer with 20% dropped out neuron-To reduce overfitting!


       
    def forward(self, img):
        flattened = img.view(-1, 512)
        activation1 = self.layer1(flattened)
        activation1 = F.relu(activation1)
        activation2 = self.dropout(self.layer2(activation1))
        # activation2 = F.relu(activation2)
        # activation3 = self.layer3(activation2)

        activation2 = activation2.squeeze(1) # Flatten to [batch_size]


        return activation2




#LOAD THE SAVED MODEL

saved_model = Classifier_withDropout() #INSTANTIATE
saved_model.load_state_dict((torch.load("model_TransferLearning_GoogleNet_bs128_lr0.001_epoch10")))#LOAD
saved_model.eval() #MUST TO REMOVE DROPOUT
saved_model=saved_model.cuda()#GPU 





In [ ]:
#LOOP OVER EVERY IMAGE IN TEST LOADER

for images, labels in test_loader:
       features = googlenet(images) #COMPUTE FEATURES FOR EACH IMAGE USING GOOGLE NET
       #This line is NEEDED OTHER WISE ERRORS IN BACK PROPAGATION!!
       features_tensor = torch.from_numpy(features.detach().cpu().numpy())
       output=saved_model(features_tensor)#PASS TO THE LOADED MODEL TO MAKE PREDICTION
       pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
       title=""
       if(pred==0):
          title='masked'
       else:
          title='no mask'
       plt.imshow(face_image)
       plt.title(title)